# Imports

In [ ]:
import sys
import logging
import shutil
from uuid import uuid4
from tempfile import gettempdir
from os.path import join, isdir
import folium
from aequilibrae import Parameters, logger, Project
import pandas as pd

In [2]:
from functions.country_borders import get_country_borders
from functions.map_path import map_single_path
from functions.map_modes import map_all_modes
from functions.country_subdivisions import get_subdivisions

# Model place (Country) to build the model for

In [3]:
model_place = 'Uzbekistan'

# Environment setup

In [4]:
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

In [5]:
fldr = f'D:/OuterLoop/OneDrive - Outer Loop Consulting/Server/Projects/2022/01-01-WorldBank GPBP/Models/{model_place}'

# if isdir(fldr):
#   shutil.rmtree(fldr)

In [6]:
project = Project()
project.open(fldr)

2022-05-24 10:03:44,203;aequilibrae;INFO ; Opened project on D:/OuterLoop/OneDrive - Outer Loop Consulting/Server/Projects/2022/01-01-WorldBank GPBP/Models/Uzbekistan


# Create Road Network from OSM

In [6]:
project = Project()
project.new(fldr)

/srv/conda/envs/saturn/lib/python3.9/site-packages/aequilibrae-0.7.3-py3.9-linux-x86_64.egg/aequilibrae/parameters.py:53: UserWarning: No pre-existing parameter file exists for this project. Will use default
  warn('No pre-existing parameter file exists for this project. Will use default')


2022-05-20 04:35:46,695;aequilibrae;INFO ; Created project on ../../Andorra


In [7]:
new_fields = [{'bridge': {'description': 'bridge flag', 'osm_source': 'bridge', 'type': 'text'}},
              {'tunnel': {'description': 'tunnel flag', 'osm_source': 'tunnel', 'type': 'text'}},
              {'toll': {'description': 'toll flag', 'osm_source': 'toll', 'type': 'text'}},
              {'surface': {'description': 'pavement surface', 'osm_source': 'surface', 'type': 'text'}}]

par = Parameters()
par.parameters['network']['links']['fields']['one-way'].extend(new_fields)
par.write_back()

In [8]:
%%time
project.network.create_from_osm(place_name=model_place)

2022-05-20 04:35:48,316;aequilibrae;INFO ; PLACE FOUND:Andorra
2022-05-20 04:35:48,374;aequilibrae;INFO ; Downloading data
2022-05-20 04:35:48,487;aequilibrae;DEBUG ; Downloading polygon 1 of 1
2022-05-20 04:35:58,496;aequilibrae;DEBUG ; Posting to http://overpass-api.de/api/interpreter with timeout=540, "{'data': '[out:json][timeout:540];(way["highway"]["area"!~"yes"]["highway"!~"proposed|raceway|construction|abandoned|platform"]["service"!~"parking|parking_aisle|driveway|private|emergency_access"]["access"!~"private"](42.428824,1.413578,42.655936,1.786664);>;);out;'}"
2022-05-20 04:36:02,445;aequilibrae;INFO ; Downloaded 15,985.0KB from overpass-api.de in 3.95 seconds
2022-05-20 04:36:02,936;aequilibrae;INFO ; Building Network
2022-05-20 04:36:03,004;aequilibrae;INFO ; Separating nodes and links
2022-05-20 04:36:04,044;aequilibrae;INFO ; Setting data structures for nodes
2022-05-20 04:36:04,999;aequilibrae;INFO ; Setting data structures for links
2022-05-20 04:36:05,039;aequilibrae;I

# Restricts import to the country's borders only

In [11]:
place_geo = get_country_borders(model_place)

### Adds the country's geometry to its own layer in the model

In [13]:
if place_geo.area > 0:
    project.conn.execute('CREATE TABLE IF NOT EXISTS country_borders("country_name" TEXT);')
    project.conn.execute("SELECT AddGeometryColumn( 'country_borders', 'geometry', 4326, 'MULTIPOLYGON', 'XY' );")

    project.conn.execute("SELECT CreateSpatialIndex( 'country_borders' , 'geometry' );")
    project.conn.commit()
    
    sql = 'INSERT into country_borders(country_name, geometry) VALUES(?, CastToMulti(GeomFromWKB(?, 4326)));'
    project.conn.execute(sql, [model_place, place_geo.wkb])
    project.conn.commit()

### Removes all links that are outside the country's borders

In [14]:
%%time
if place_geo.area > 0:
    sql = """DELETE from Links where link_id not in (SELECT a.link_id
    FROM links AS a, country_borders as b
    WHERE ST_Intersects(a.geometry, b.geometry) = 1)"""

    project.conn.execute(sql)
    project.conn.commit()

CPU times: total: 1min 53s
Wall time: 1min 58s


# Adds political subdivisions

In [7]:
gdf = get_subdivisions(model_place)

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: No module named 'fiona'

In [ ]:
gdf.plot()

In [ ]:
#There is no real elegant way to add a GeoDataFrame to an spatialite database
if gdf.shape[0] > 0:
    project.conn.execute('Drop TABLE IF EXISTS country_subdivisions;')
    project.conn.execute('CREATE TABLE IF NOT EXISTS country_subdivisions("country_name" TEXT, "division_name" TEXT, "level" INTEGER);')
    project.conn.execute("SELECT AddGeometryColumn( 'country_subdivisions', 'geometry', 4326, 'MULTIPOLYGON', 'XY' );")

    project.conn.execute("SELECT CreateSpatialIndex( 'country_subdivisions' , 'geometry' );")
    project.conn.commit()
    
    sql = 'INSERT into country_subdivisions(country_name, division_name, level, geometry) VALUES(?, ?, ?, CastToMulti(GeomFromWKB(?, 4326)));'
    for _, rec in gdf.iterrows():
        project.conn.execute(sql, [model_place, rec['name'], rec.level, rec.geometry.wkb])
    project.conn.commit()

# Displays the network

# We test how would path computation work

In [15]:
from aequilibrae.paths import PathResults

In [16]:
# This network manipulation is just temporary. We will work in more detail when doing some proper network modelling.
project.conn.execute('update nodes set is_centroid=1')
project.conn.commit()

project.network.build_graphs()
graph = project.network.graphs['c']

# let's say we want to minimize distance
graph.set_graph('distance')

# And will skim time and distance while we are at it
graph.set_skimming(['distance'])

graph.set_blocked_centroid_flows(False)

2022-05-20 04:37:17,180;aequilibrae;WARNING ; Field(s) speed, capacity, lanes has(ve) at least one NaN value. Check your computations
2022-05-20 04:37:17,308;aequilibrae;WARNING ; Field(s) speed, capacity, lanes has(ve) at least one NaN value. Check your computations
2022-05-20 04:37:17,441;aequilibrae;WARNING ; Field(s) speed, capacity, lanes has(ve) at least one NaN value. Check your computations
2022-05-20 04:37:17,575;aequilibrae;WARNING ; Field(s) speed, capacity, lanes has(ve) at least one NaN value. Check your computations


In [17]:
res = PathResults()
res.prepare(graph)

sql = 'select node_id, modes from nodes where modes like "%c%"'
df_nodes = pd.read_sql(sql, project.conn)

# Let's get random origin and destination for which a path exist so we can map it
while True:
    o,d = df_nodes.node_id.sample(2).values
    res.compute_path(o, d)
    if res.path.shape[0]:
        break

In [18]:
map_single_path(project, res)

In [19]:
project.close()

2022-05-20 04:37:18,977;aequilibrae;INFO ; Closed project on ../../Andorra
